# Trova la Moneta Falsa

In un numpy array di n elementi (2 < n < 240) ci sono tutte monete di un certo peso ed una leggermente più pesante. Dovete scoprire quale è l'indice della moneta più pesante!
Per farlo dovete costruire una funzione TEST che dati due array confronti le loro somme e restituisca:
- 1 se il primo è più pesante
- 2 se il secondo è più pesante
- 0 se hanno lo stesso peso

Questa funzione è l'unico modo in cui potete accedere agli elementi dell'array ed il vostro obiettivo è scoprire la moneta falsa in meno TEST possibili.

Il programma dovrà generare nel main una serie di numpy array di varie dimensione e ogni elemento inizializzato ad un numero casuale tra 10000 e 20000 e successivamente scgeliere un indice casuale e aggiungere 1 a quell'elemento.
Create poi una matrice contenente:
- Numero di elementi dell'i-esimo array
- Indice della moneta più pesante dell'i-esimo array
- Indice restituito dal vostro algoritmo dell'i-esimo array
- Numero di volte in cui avete usato TEST per l'i-esimo array

La valutazione terrà conto dell'ottimalità e dell'esattezza della soluzione in primo luogo, successivamente dello stile e chiarezza del codice.
Non è ammesso l'utilizzo di liste, tuple o simili, ma solo di array di numpy.
Buon lavoro!

EXTRA BONUS: fatelo nel caso in cui non sapete se la moneta falsa è più pesante o leggera

In [1]:
import numpy
import random

In [2]:
##### FUNZIONE TEST #####

def TEST(array1, array2):
    sum1 = numpy.sum(array1)
    sum2 = numpy.sum(array2)
    return 1*(sum1>sum2) + 2*(sum1<sum2)

In [3]:
##### DESCRIZIONE DELL'ALGORITMO #####
# A partire da un'osservazione qualitativa (e dall'esperienza dei risolutori costruita sui videogiochi de 
# "Il professore Layton") si nota che per ottenere un algoritmo ottimale (ossia che minimizzi il numero di iterazioni nei casi 
# peggiori) si parte studiando la modularità di n (numero di elementi dell'array) rispetto a 3:
# 1) Qualora il numero sia multiplo di 3 allora si divide il vettore in 3 parti e si confrontano 2 di esse. Se una di loro è più
#    pesante la si terrà per procedere mentre se sono uguali significa che la moneta pesante sta nel terzo gruppo.
# 2) Se invece n non è multiplo di 3, si divide l'array in 2 gruppi (tenendo una moneta spaiata qualora n non si divisibile
#    nemmeno per 2) e si confrontano. Se sono diversi si mantiene il più pesante, se sono uguali significa che la moneta pesante
#    è proprio quella spaiata e si può uscire dall'algoritmo con il risultato trovato "in anticipo" rispetto al caso peggiore. 
# 3) Ripetere dal punto 1 con il gruppo di monete che si è tenuto (funzione ricorsiva)

##### ALCUNE ANNOTAZIONI #####
# 1) Tra i casi piccoli analizzati manualmente si è notato che il numero 15 sfugge alla regola nel senso che sia partendo con la 
#    divisione in 3 gruppi che con quella in 2, il caso peggiore è equivalente. Di conseguenza iniziare con la divisione per 2 
#    sarebbe preferibile dal momento che potrebbe condurre ad una uscita anticipata dall'algoritmo. 
#    Tuttavia per una maggior chiarezza e leggibilità del programma e siccome la probabilità che si esca in anticipo dal caso 
#    n=15 sono piuttosto irrisorie, si è preferito ignorare il fatto. 
# 2) La tecnica utilizzata per tenere traccia dell'indice della moneta più pesante è stata quella di sommare ad ogni chiamata 
#    della funzione ricorsiva (fine_heavier_coin) l'indice che indica il primo elemento di ogni sotto-array generato dallo split
#    Così, ritornando di volta in volta dalla chiamata più profonda a quella più superficiale, si ricostruisce la posizione 
#    della moneta pesante nell'array di partenza


def find_heavier_coin(array):                    
    
    dim = numpy.size(array)                      ## Numero di elementi 
    
    if dim<=3:                                   ## Se il vettore contiene meno di 3 elementi allora siamo arrivati a poter 
                                                 ## trovare la soluzione
        if TEST(array[0],array[1]) == 1:
            return (0,1)
        elif TEST(array[0],array[1]) == 2:
            return (1,1)
        else:
            return (2,1)
        
    if dim%3 == 0:
        arr0 = numpy.array(numpy.split(array,3)[0])
        arr1 = numpy.array(numpy.split(array,3)[1])
        arr2 = numpy.array(numpy.split(array,3)[2])
        if TEST(arr0,arr1) == 1:
            (index,counter) = (find_heavier_coin(arr0)[0] + 0 , find_heavier_coin(arr0)[1] + 1)
        
        elif TEST(arr0,arr1) == 2:
            (index,counter) = (find_heavier_coin(arr1)[0] + dim/3 , find_heavier_coin(arr1)[1] + 1)
        
        else: 
            (index,counter) = (find_heavier_coin(arr2)[0] + 2*dim/3 , find_heavier_coin(arr2)[1] + 1)
    
    else:
        if dim%2 == 0:
            arr0 = numpy.array(numpy.split(array,2)[0])
            arr1 = numpy.array(numpy.split(array,2)[1])
            if TEST(arr0,arr1) == 1:
                (index,counter) = (find_heavier_coin(arr0)[0] + 0 , find_heavier_coin(arr0)[1] + 1)
            else: 
                (index,counter) = (find_heavier_coin(arr1)[0] + dim/2 , find_heavier_coin(arr1)[1] + 1)
        
        else:
            arr0 = numpy.array(numpy.split(array, (int((dim-1)/2) , dim-1))[0])
            arr1 = numpy.array(numpy.split(array, (int((dim-1)/2) , dim-1))[1])
            if TEST(arr0,arr1) == 1:
                (index,counter) = (find_heavier_coin(arr0)[0] + 0 , find_heavier_coin(arr0)[1] + 1)
            elif TEST(arr0,arr1) == 2:
                (index,counter) = (find_heavier_coin(arr1)[0] + (dim-1)/2 , find_heavier_coin(arr1)[1] + 1)
            else:
                return (dim-1 , 1)
    
    return (index,counter)

In [4]:
##### MAIN #####

max_arrays = 100                                   ## Quanti array si vogliono creare di cui poi cercare l'elemento più pesante
matrix = numpy.zeros((max_arrays,4))
for i in range(0,max_arrays):
    n = random.randint(3,239)                      ## Lunghezza array
    
    val = random.randint(10000,20000)              ## Per comodità e immediatezza di lettura ho creato i pesi come interi.
    
    ## random.seed(1)
    ## val= 10000*numpy.random.random() + 10000    ## Qualora si volessero pesi non interi si possono usare questi comandi
   
    array = numpy.array(numpy.full((1,n), val)[0])
    pos_heavier = random.randint(0,n-1)            ## Posizione elemento più pesante
    array[pos_heavier] += 1                        ## Aumento peso
    
    (index, counter) = find_heavier_coin(array)    ## funzione che trova la moneta più pesante
    
    matrix[i,0] = n                                ## compilazione della matrice di resoconto ##
    matrix[i,1] = pos_heavier                      ##                                         ##
    matrix[i,2] = index                            ##                                         ##
    matrix[i,3] = counter                          ##                                         ##

print(matrix)

[[ 76.  63.  63.   5.]
 [ 89.  83.  83.   6.]
 [ 78.  61.  61.   5.]
 [ 15.  13.  13.   3.]
 [195.  45.  45.   7.]
 [154.  83.  83.   6.]
 [ 90.  45.  45.   5.]
 [118.   1.   1.   6.]
 [181. 155. 155.   6.]
 [196. 184. 184.   7.]
 [ 57.  56.  56.   2.]
 [158. 143. 143.   4.]
 [ 67.  54.  54.   3.]
 [198. 161. 161.   6.]
 [134.   3.   3.   6.]
 [172.   5.   5.   6.]
 [ 35.   1.   1.   5.]
 [220.  83.  83.   6.]
 [210. 194. 194.   7.]
 [205.  62.  62.   7.]
 [ 14.  10.  10.   3.]
 [ 24.  21.  21.   4.]
 [  6.   3.   3.   2.]
 [ 32.   6.   6.   5.]
 [213.   5.   5.   7.]
 [104.  46.  46.   6.]
 [239.  61.  61.   7.]
 [229. 139. 139.   6.]
 [190. 119. 119.   7.]
 [101.  17.  17.   6.]
 [125.  33.  33.   6.]
 [232.  76.  76.   7.]
 [207.  67.  67.   4.]
 [238. 134. 134.   7.]
 [ 35.  28.  28.   5.]
 [162.  50.  50.   5.]
 [232.  46.  46.   7.]
 [179.  93.  93.   6.]
 [100.   6.   6.   6.]
 [ 70.  14.  14.   6.]
 [ 17.   0.   0.   4.]
 [105.  36.  36.   6.]
 [102.  38.  38.   6.]
 [ 33.   3.